<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Train_38.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd "/content/Dataset(s)/food-101-img_txt/food101"

/content/Dataset(s)/food-101-img_txt/food101


# Training Loop

Create Text and Image txt Files

In [ ]:
# def custom_sort(line):
#     # Extract the filename and numeric part
#     filename, text = map(str.strip, line.split(',', 1))
#     alpha_part, numeric_part = filename[:-4].rsplit('_', 1)  # Remove the ".txt" extension

#     # Sort alphabetically first and then numerically
#     return (alpha_part, numeric_part)

# # Read the contents of the file into a list
# with open('./train_text.txt', 'r') as file:
#     lines = file.readlines()

# # Sort the lines using the custom sorting function
# sorted_lines = sorted(lines, key=custom_sort)

# # Write the sorted lines back to the file
# with open('./train_text_sorted.txt', 'w') as file:
#     file.writelines(sorted_lines)


In [ ]:
# #Create txt file for Image Paths and Labels
# import os

# dataset_folder = '/content/Dataset(s)/food-101-img_txt/food101/food101/train/'
# output_file = './food101/train_dataset.txt'

# def custom_sort_key(item):
#     return item.replace('@', '')

# with open(output_file, 'w') as f:
#     for label in sorted(os.listdir(dataset_folder)):
#         label_path = os.path.join(dataset_folder, label)
#         if os.path.isdir(label_path):
#             class_name = label
#             image_files = sorted(os.listdir(label_path))
#             for image_file in image_files:
#                 image_path = os.path.join(label_path, image_file)
#                 f.write(f'{image_path}, {class_name}\n')

Load Image Data From Label Files

In [2]:
import os

image_file_paths = []
image_genre_labels = []

labels_file = "./food101/train_dataset.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()
        image_file_paths.append(filename)
        image_genre_labels.append(labels)

In [5]:
print(len(image_file_paths))
print(len(image_genre_labels))

67988
67988


In [4]:
import os

test_image_file_paths = []
test_image_genre_labels = []

labels_file = "./food101/test_dataset.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()
        test_image_file_paths.append(filename)
        test_image_genre_labels.append(labels)

In [6]:
print(len(test_image_file_paths))
print(len(test_image_genre_labels))

22716
22716


Load Text Data From CSV

In [7]:
file_path = "./test_text_sorted.txt"

test_texts_csv_lst = []

with open(file_path, 'r') as file:
    for line in file:
        _, text = map(str.strip, line.split(',', 1))
        test_texts_csv_lst.append(text)

In [8]:
file_path = "./train_text_sorted.txt"

texts_csv_lst = []

with open(file_path, 'r') as file:
    for line in file:
        _, text = map(str.strip, line.split(',', 1))
        texts_csv_lst.append(text)

In [9]:
len(texts_csv_lst) , len(test_texts_csv_lst)

(67988, 22716)

Label Encoder

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(image_genre_labels)

LabelEncoder()

In [11]:
transformed_genre_labels = label_encoder.transform(image_genre_labels)
test_transformed_genre_labels = label_encoder.transform(test_image_genre_labels)

In [12]:
transformed_genre_labels = transformed_genre_labels.tolist()
test_transformed_genre_labels = test_transformed_genre_labels.tolist()

In [13]:
len(transformed_genre_labels), len(test_transformed_genre_labels)

(67988, 22716)

Create Dataset

In [14]:
image_file_paths[12000], texts_csv_lst[12000], transformed_genre_labels[12000]

('/content/Dataset(s)/food-101-img_txt/food101/food101/train/cheesecake/cheesecake_783.jpg',
 '8 Cheesecake Recipes We Love | Serious Eats: Sweets,cheesecake',
 17)

In [16]:
test_image_file_paths[0], test_texts_csv_lst[0], test_transformed_genre_labels[0]

('/content/Dataset(s)/food-101-img_txt/food101/food101/test/apple_pie/apple_pie_1.jpg',
 'Apple Pie and Cake Recipes - Health.com,apple_pie',
 0)

In [17]:
import torch
from PIL import Image as pil

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, texts, labels, processor):
        self.images = images
        self.texts = texts
        self.labels = labels
        self.processor = processor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.processor(pil.open(image).convert("RGB"), text,  max_length=32, add_special_tokens=True,  is_split_into_words=False, padding="max_length", truncation=True, return_tensors="pt")
        for k,v in encoding.items():
          encoding[k] = v.squeeze()

        # encoding["labels"] = label
        encoding["labels"] = torch.tensor(label)

        return encoding

In [18]:
from transformers import ViltProcessor

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-nlvr2")

dataset = CustomDataset(images=image_file_paths,
                     texts=texts_csv_lst,
                     labels=transformed_genre_labels,
                     processor=processor)

test_dataset = CustomDataset(images=test_image_file_paths,
                     texts=test_texts_csv_lst,
                     labels=test_transformed_genre_labels,
                     processor=processor)

In [19]:
dataset[0]

{'input_ids': tensor([  101,  6207, 11345,  1011, 16948,  1010,  6207,  1035, 11345,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), 'pixel_values': tensor([[[ 0.4824,  0.3333,  0.2863,  ...,  0.4353,  0.4353,  0.4353],
         [ 0.3647,  0.2863,  0.2941,  ...,  0.4353,  0.4510,  0.4510],
         [ 0.3098,  0.2863,  0.3020,  ...,  0.4588,  0.4588,  0.4510],
         ...,
         [ 0.1373,  0.1608,  0.2314,  ...,  0.1686,  0.1686,  0.2627],
         [ 0.2078,  0.1451,  0.2000,  ...,  0.3176,  0.1608,  0.1373],
         [ 0.2706,  0.2627,  0.1529,  ...,  0.1922,  0.3020,  0.2157]],

      

In [20]:
test_dataset[0]

{'input_ids': tensor([  101,  6207, 11345,  1998,  9850, 19328,  1011,  2740,  1012,  4012,
         1010,  6207,  1035, 11345,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), 'pixel_values': tensor([[[-0.0431, -0.0431, -0.0431,  ...,  0.3961,  0.3569,  0.3647],
         [-0.1529, -0.1451, -0.1451,  ...,  0.4902,  0.3490,  0.2863],
         [-0.2235, -0.2235, -0.2157,  ...,  0.4824,  0.3098,  0.2627],
         ...,
         [ 0.2000,  0.2000,  0.2000,  ...,  0.4902,  0.4588,  0.4275],
         [ 0.1922,  0.1922,  0.1922,  ...,  0.4902,  0.4588,  0.4431],
         [ 0.1922,  0.1922,  0.1922,  ...,  0.4667,  0.4353,  0.4196]],

      

Collate_fun and Data Loader

In [21]:
import torch

def collate_fn(batch):
  input_ids = [item['input_ids'] for item in batch]
  pixel_values = [item['pixel_values'] for item in batch]
  labels = [item['labels'] for item in batch]

  encoding = processor.image_processor.pad(pixel_values, return_tensors="pt")

  batch = {}
  batch['input_ids'] = torch.stack(input_ids)
  batch['pixel_values'] = encoding['pixel_values']
  batch['labels'] = torch.stack(labels)

  return batch

  # return {
  #     "input_ids": torch.stack([x["input_ids"] for x in batch]),
  #     "pixel_values": torch.stack([x["pixel_values"] for x in encoding]),
  #     # "attention_mask": torch.stack([x["attention_mask"] for x in batch]),
  #     # "token_type_ids": torch.stack([x["token_type_ids"] for x in batch]),
  #     # "pixel_mask": torch.stack([x["pixel_mask"] for x in batch]),
  #     "labels": torch.stack([torch.tensor(x["labels"]) for x in batch]),
  # }

In [22]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=12, shuffle=True)
test_dataloader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=12, shuffle=False)

Check Dimensions

In [23]:
batch = next(iter(train_dataloader))
test_batch = next(iter(test_dataloader))

In [24]:
for k,v in batch.items():
  print(k, v.shape)

input_ids torch.Size([12, 32])
pixel_values torch.Size([12, 3, 544, 608])
labels torch.Size([12])


In [25]:
for k,v in test_batch.items():
  print(k, v.shape)

input_ids torch.Size([12, 32])
pixel_values torch.Size([12, 3, 608, 608])
labels torch.Size([12])


In [ ]:
# processor.decode(batch["input_ids"][0])

In [ ]:
# batch["pixel_values"][0]

import Model and Processor

In [ ]:
# label2id = {label: encoded_id for label, encoded_id in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
# id2label = {encoded_id: label for encoded_id, label in enumerate(label_encoder.classes_)}

In [ ]:
# label2id

In [26]:
valid_labels = label_encoder.classes_.tolist()

In [ ]:
# valid_labels

In [27]:
from transformers import ViltForImagesAndTextClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViltForImagesAndTextClassification.from_pretrained("dandelin/vilt-b32-finetuned-nlvr2",
                                                                label2id = {label: str(i) for i, label in enumerate(valid_labels)},
                                                                id2label = {str(i): label for i, label in enumerate(valid_labels)},
                                                                num_images = 1,
                                                                ignore_mismatched_sizes=True)

model.to(device)

Some weights of ViltForImagesAndTextClassification were not initialized from the model checkpoint at dandelin/vilt-b32-finetuned-nlvr2 and are newly initialized because the shapes did not match:
- classifier.0.weight: found shape torch.Size([1536, 1536]) in the checkpoint and torch.Size([768, 768]) in the model instantiated
- classifier.0.bias: found shape torch.Size([1536]) in the checkpoint and torch.Size([768]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1536]) in the checkpoint and torch.Size([768]) in the model instantiated
- classifier.1.bias: found shape torch.Size([1536]) in the checkpoint and torch.Size([768]) in the model instantiated
- classifier.3.weight: found shape torch.Size([2, 1536]) in the checkpoint and torch.Size([101, 768]) in the model instantiated
- classifier.3.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([101]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to us

ViltForImagesAndTextClassification(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(3, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_f

In [28]:
model.config

ViltConfig {
  "_name_or_path": "dandelin/vilt-b32-finetuned-nlvr2",
  "architectures": [
    "ViltForImagesAndTextClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "10": "bruschetta",
    "100": "waffles",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "2": "baklava",
    "20": "chicken_wings",
    "21": "chocolate_cake",
    "22": "chocolate_mousse",
    "23": "churros",
    "24": "clam_chowder",
    "25": "club_sandwich",
    "26": "crab_cakes",
    "27": "creme_brulee",
    "28": "croque_madame",
    "29": "cup_cakes",
    "3": "beef_carpaccio",
    "30": "deviled_eggs",
    "31": "donuts",
    "32": "dumplings",
    "33": "edamame",
    "34": "eg

Training Loop

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tqdm.notebook import tqdm
import numpy as np
from datetime import datetime

In [30]:
checkpoint_dir = "/content/Model/Models-Train-38/"

In [31]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(50):
   model.train()
   print(f"Epoch: {epoch}")
   train_loss = 0
   for batch in tqdm(train_dataloader):
        # get the inputs;
        batch = {k:v.to(device) for k,v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        # print("Loss:", loss.item())
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

   print("Training Loss: ", train_loss)
   print(f"Saving Model at epoch: {epoch}")

   timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
   model_folder = os.path.join(checkpoint_dir, f"model_{timestamp}")
   os.makedirs(model_folder, exist_ok=True)

  #  processor.save_pretrained(checkpoint_dir)
   model.save_pretrained(model_folder)

   model.eval()
   preds = []
   orignal = []
   for batch in tqdm(test_dataloader):
       batch = {k:v.to(device) for k,v in batch.items()}

       outputs = model(**batch)

       scores = outputs.logits
       preds_current = torch.argmax(scores, dim=1)

       preds += preds_current.cpu().numpy().tolist()
       orignal += batch["labels"].detach().cpu().numpy().tolist()

   accuracy = accuracy_score(orignal, preds)
   precision = precision_score(orignal, preds, average='macro')
   recall = recall_score(orignal, preds, average='macro')
   f1 = f1_score(orignal, preds, average='macro')

   print(f"Accuracy: {accuracy}")
   print(f"Precision: {precision}")
   print(f"Recall: {recall}")
   print(f"F1-Score: {f1}")


Epoch: 0


  0%|          | 0/5666 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Training Loss:  1127.4939217668725
Saving Model at epoch: 0


  0%|          | 0/1893 [00:00<?, ?it/s]

Accuracy: 0.9916358513822856
Precision: 0.9921931645762674
Recall: 0.9915688183273522
F1-Score: 0.9916669351842866
Epoch: 1


  0%|          | 0/5666 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Missing Percentage Validation

In [ ]:
import os

test_image_file_paths = []
test_image_genre_labels = []

labels_file = "./ferramenta_Images/test_dataset.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()
        test_image_file_paths.append(filename)
        test_image_genre_labels.append(labels)

In [ ]:
len(test_image_file_paths)

In [ ]:
import pandas as pd

df_test = pd.read_csv("./test.csv")

In [ ]:
df_test

In [ ]:
# null_indices = df_test[df_test['description'].isnull()].index


# print("Indices of rows with null entries in 'text' column:")
# print(null_indices)


In [ ]:
# As description contains 1024 null entries
df_test['description'].fillna(" ", inplace=True)

In [ ]:
df_test['text'] = df_test["title"] + df_test['description']

In [ ]:
import re

def remove_numbers(text):
    if isinstance(text, str):
        return re.sub(r'\d+', '', text)
    else:
        return text

In [ ]:
df_test['text'] = df_test['text'].apply(remove_numbers)

In [ ]:
null_text_count = df_test['text'].isnull().sum()
print(null_text_count)

In [ ]:
df_test.drop(df_test.columns.difference(['class', 'text']), axis=1, inplace=True)

In [ ]:
df_test['class'] = df_test['class'].str.replace('_@', '')

In [ ]:
unique_classes = df_test['class'].unique()
print(len(unique_classes))

In [ ]:
# class_counts = df_test['class'].value_counts()


# print("Class counts:")
# print(class_counts)

In [ ]:
# percentage_to_filter = 0.9  # Change this to your desired percentage

In [ ]:
# filtered_rows = pd.DataFrame()

# for class_name in unique_classes:
#     class_rows = df_test[df_test['class'] == class_name]
#     num_rows_to_keep = int(len(class_rows) * percentage_to_filter)
#     sampled_rows = class_rows.sample(n=num_rows_to_keep, random_state=42)  # You can change the random_state if needed
#     filtered_rows = pd.concat([filtered_rows, sampled_rows])

# remaining_rows = df_test[~df_test.index.isin(filtered_rows.index)]
# remaining_rows['text'] = " "


# df_test = pd.concat([filtered_rows, remaining_rows])

In [ ]:
percentage_to_keep = 1

final_df = pd.DataFrame(columns=df_test.columns)

for class_name in unique_classes:

    class_rows = df_test[df_test['class'] == class_name]
    num_rows_to_keep = int(len(class_rows) * percentage_to_keep)
    sampled_rows = class_rows.head(num_rows_to_keep)
    final_df = pd.concat([final_df, sampled_rows])

remaining_rows = df_test[~df_test.index.isin(final_df.index)]
remaining_rows['text'] = " "

# Step 6: Combine the sampled and updated rows to get the final DataFrame
final_df_updated = pd.concat([final_df, remaining_rows])

In [ ]:
df_test = final_df_updated

In [ ]:
# class_counts = final_df['class'].value_counts()

# # Display the counts
# print("Class counts:")
# print(class_counts)

In [ ]:
# Check how many rows in df_test have "no text" in the 'text' column
count_no_text = (df_test['text'] == " ").sum()
print(f"Number of rows with 'no text': {count_no_text}")

In [ ]:
# df_test

In [ ]:
# df_test[ df_test['class']== "estrattore_"]

In [ ]:
test_texts_csv_lst = df_test['text'].tolist()

In [ ]:
len(test_texts_csv_lst)

In [ ]:
# from transformers import AutoTokenizer

# # Assuming df_test is your DataFrame with 'text' column

# model_name = "dbmdz/bert-base-italian-cased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Tokenize the 'text' column
# df_test['text_tokens'] = df_test['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, padding=True))

In [ ]:
# print(tokenizer.decode(df_test['text_tokens'][0]))
# print(df_test['text_tokens'][0])

In [ ]:
# test_texts_csv_lst[0]

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(test_image_genre_labels)
test_transformed_genre_labels = label_encoder.transform(test_image_genre_labels)
test_transformed_genre_labels = test_transformed_genre_labels.tolist()

In [ ]:
import torch
from PIL import Image as pil

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, texts, labels, processor):
        self.images = images
        self.texts = texts
        self.labels = labels
        self.processor = processor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.processor(pil.open(image).convert("RGB"), text,  max_length=32, add_special_tokens=True,  is_split_into_words=False, padding="max_length", truncation=True, return_tensors="pt")
        for k,v in encoding.items():
          encoding[k] = v.squeeze()

        # encoding["labels"] = label
        encoding["labels"] = torch.tensor(label)

        return encoding

In [ ]:
from transformers import ViltProcessor

processor = ViltProcessor.from_pretrained("/content/Model/Models-Train-35/")

test_dataset = CustomDataset(images=test_image_file_paths,
                     texts=test_texts_csv_lst,
                     labels=test_transformed_genre_labels,
                     processor=processor)

In [ ]:
import torch

def collate_fn(batch):
  return {
      "input_ids": torch.stack([x["input_ids"] for x in batch]),
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      # "attention_mask": torch.stack([x["attention_mask"] for x in batch]),
      # "token_type_ids": torch.stack([x["token_type_ids"] for x in batch]),
      # "pixel_mask": torch.stack([x["pixel_mask"] for x in batch]),
      "labels": torch.stack([torch.tensor(x["labels"]) for x in batch]),
  }

In [ ]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=12, shuffle=False)

# Fused Validation

In [ ]:
from transformers import ViltForImagesAndTextClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViltForImagesAndTextClassification.from_pretrained("/content/Model/Models-Train-35/")

model.to(device)

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

preds = []
orignal = []

for batch in tqdm(test_dataloader):
    batch = {k:v.to(device) for k,v in batch.items()}

    outputs = model(**batch)

    scores = outputs.logits
    preds_current = torch.argmax(scores, dim=1)

    preds += preds_current.cpu().numpy().tolist()
    orignal += batch["labels"].detach().cpu().numpy().tolist()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

accuracy = accuracy_score(orignal, preds)
precision = precision_score(orignal, preds, average='macro')
recall = recall_score(orignal, preds, average='macro')
f1 = f1_score(orignal, preds, average='macro')

In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

# Extract Model's Number of Trained Parameters

In [ ]:
# from transformers import ViltForImagesAndTextClassification

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = ViltForImagesAndTextClassification.from_pretrained("/content/Model/Models-Train-35/")

# model.to(device)

In [ ]:
from transformers import ViTForImageClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-18/checkpoint-81000").to(device)

num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
print(num_parameters)